In [5]:
%matplotlib inline

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision
import math
from torch.autograd import Variable
import copy
import torchvision.transforms as transforms

In [7]:
import pandas as pd 
import numpy as np

In [8]:
rating_dir = './RCdata/rating_final.csv'

In [212]:
rest_deets_dir = './RCdata/geoplaces2.csv'

In [9]:
ratings = pd.read_csv(rating_dir)

In [214]:
rest_deets = pd.read_csv(rest_deets_dir, encoding = 'cp1252')

In [215]:
rest_deets

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none
5,135040,22.135617,-100.969709,0101000020957F00001B552189B84A58C15A2AAEFD2CA2...,Restaurant los Compadres,Camino a Simon Diaz 155 Centro,San Luis Potosi,SLP,Mexico,?,...,Wine-Beer,none,informal,no_accessibility,high,?,familiar,f,closed,none
6,132732,23.754357,-99.171288,0101000020957F00008A20E615808157C16272FECBF84F...,Taqueria EL amigo,Calle Mezquite Fracc Framboyanes,Cd Victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,none,casual,completely,low,?,familiar,f,open,none
7,132875,22.149901,-100.993779,0101000020957F00008A2A0747DE4758C11EB31D2A31A8...,shi ro ie,?,?,?,?,?,...,Wine-Beer,section,informal,no_accessibility,high,?,familiar,t,open,Internet
8,132609,23.760268,-99.165865,0101000020957F0000A478418BBA8057C133851EB22C4E...,Pollo_Frito_Buenos_Aires,tampico,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,not permitted,informal,completely,low,?,quiet,t,closed,none
9,135082,22.151448,-100.915099,0101000020957F0000A29FAF95CD4958C1FEEEBB73A991...,la Estrella de Dimas,Villa de Pozos 192 Villa de Pozos,San Luis Potosi,SLP,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,?,familiar,f,closed,none


In [219]:
rest_deets.loc[rest_deets['placeID'] == 134999]['name'][0]

'Kiku Cuernavaca'

In [13]:
from sklearn.model_selection import train_test_split
rating_train, rating_test = train_test_split(ratings, test_size=0.3)

In [179]:
n_users = len(ratings.userID.unique())
n_items = len(ratings.placeID.unique())

In [180]:
n_users
#n_items

138

In [190]:
user_list = np.array(ratings.userID.unique())
item_list = np.array(ratings.placeID.unique())

In [ ]:
#use ratings = food + service rating put together

In [17]:
ratings_mat = np.zeros((n_users, n_items))
food_mat = np.zeros((n_users, n_items))
service_mat = np.zeros((n_users, n_items))

In [15]:
#ratings_mat = ratings_mat.tolist()

In [16]:
#ratings_mat.dim

In [31]:
ratings['userID'].iloc[0]

'U1077'

In [182]:
for i in range(len(ratings)):
    user, = np.where(users == ratings['userID'].iloc[i])
    item, = np.where(items == ratings['placeID'].iloc[i])
    ratings_mat[user[0]][item[0]] = ratings['food_rating'].iloc[i] + ratings['service_rating'].iloc[i]
    #food_mat[user[0]][item[0]] = ratings['food_rating'].iloc[i]
    #service_mat[user[0]][item[0]] = ratings['service_rating'].iloc[i]

In [183]:
from scipy.sparse import rand as sprand
from scipy.sparse import lil_matrix

In [184]:
interactions_full = lil_matrix((n_users, n_items), dtype = float)
for row in ratings.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_full[user[0], item[0]] = row[4] + row[5]                  

In [185]:
interactions_full

<138x130 sparse matrix of type '<class 'numpy.float64'>'
	with 949 stored elements in LInked List format>

In [186]:
interactions_train = lil_matrix((n_users, n_items), dtype = float)
for row in rating_train.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_train[user[0], item[0]] = row[4] + row[5]                  

In [187]:
interactions_test = lil_matrix((n_users, n_items), dtype = float)
for row in rating_test.itertuples():
    user, = np.where(users == row[1])
    item, = np.where(items == row[2])
    interactions_test[user[0], item[0]] = row[4] + row[5] 

In [ ]:
#only use the ratings right now

In [23]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items,n_factors,sparse=False)
                                               

    # For convenience when we want to predict a sinble user-item pair.
    def predict(self, user, item):
        # Need to fit bias factors
        return (self.user_factors(user) * self.item_factors(item)).sum(1)
    
    # Much more efficient batch operator. This should be used for training purposes
    
    def forward(self, users, items):
        #return (self.user_factors(user) * self.item_factors(item)).sum(1)
    
        return torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))

In [59]:
class BiasedMatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items,mean, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, n_factors,sparse=False)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse = False)
        
        self.item_biases = torch.nn.Embedding(n_items,1, sparse = False)
                                               
        self.mu = mean
    
    #def forward(self, user, item):
        #pred = self.user_biases(user) + self.item_biases(item)
        #pred += (self.user_factors(user) * self.item_factors(item)).sum(1)
        #return pred
    
    def forward(self, users, items):
        
        item_means = []
        user_means = []
        for i in range(len(users)):
            item_means.append(torch.t(self.item_biases(items)))
            
        for j in range(len(items)):
            user_means.append(torch.t(self.user_biases(users)))
        
        it_means = torch.cat(item_means, 0)
        us_means = torch.cat(user_means, 0)
        us_means = torch.t(us_means)
        
        orig = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        
        y = torch.add(orig, it_means)
        
        f = torch.add(y,us_means)
        
        result = torch.add(f,self.mu)
        
        return result

In [25]:
def get_batch(batch_size,ratings):
    # Sort our data and scramble it
    rows, cols = ratings.shape
    p = np.random.permutation(rows)
    
    # create batches
    sindex = 0
    eindex = batch_size
    while eindex < rows:
        batch = p[sindex:eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch

    if eindex >= rows:
        batch = range(sindex,rows)
        yield batch

In [156]:
def full_batch(iterable):
    l = iterable.shape[0]
    #for ndx in range(0, l, n):
    print (l)
    yield iterable[0:l]

In [159]:
range(138)

range(130, 138)


In [26]:
def load_model(model,load_path):
    load_dict = torch.load(load_path)
    val_loss = load_dict['val_loss']
    model.load_state_dict(load_dict['model_state_dict'])


def checkpoint_model(val_loss, model,save_path):
    save_dict = dict(
                     val_loss=val_loss,
                     model_state_dict=model.state_dict())
                     #opt_state_dict=predictions.state_dict())
    torch.save(save_dict, save_path)

In [27]:
def run_validation(model, test_ratings, BATCH_SIZE, load_path):
    load_model(model, load_path)
    running_loss = 0.0
    loss_func = torch.nn.MSELoss()
    
    for i,batch in enumerate(get_batch(BATCH_SIZE, test_ratings)):
        
        interactions = Variable(torch.FloatTensor(test_ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(test_ratings.shape[1])))
        
        predictions = model(rows, cols)
        loss = loss_func(predictions, interactions)
        
        running_loss += np.sqrt(loss.data[0])*BATCH_SIZE
    
    epoch_loss = running_loss/test_ratings.shape[0]
    return epoch_loss

In [171]:
def generate_predictions(model, interactions_full, load_path):
    load_model(model, load_path)
    #final_top5 = [0 for i in range(5)]
    
    #for i,batch in enumerate(range(interactions_full.shape[0])):
    batch = range(interactions_full.shape[0])
    interactions = Variable(torch.FloatTensor(interactions_full[batch,:].toarray()))
    rows = Variable(torch.LongTensor(batch))
    cols = Variable(torch.LongTensor(np.arange(interactions_full.shape[1])))
    predictions = model(rows, cols)
    predictions[interactions!=0] = 0
        
    val, index = torch.sort(predictions,1, descending = True)
        
    top5 = index[:,:5]
        
    #if step == 0:
        #final = top5
            #user_ids = rows
        
    #else:
    #final = torch.cat((final, top5), 0)
            #user_ids = torch.cat((user_ids, rows),0)
    
    return top5

In [243]:
def give_top5(recs, name_list, this_user, user_list, rest_list):
    
    #rest_deets.loc[rest_deets['placeID'] == 134999]['name'][0]
    
    
    user, = np.where(user_list == this_user)
    #top5 = recs[user[0]]
    your_rests  = []
    for i in range(5):
        this_rest = recs[user[0]][i].data[0]
        rest = rest_list[this_rest]
        x = name_list.loc[name_list['placeID']== rest]
        if len(x) == 1:
            name = x['name'].item()
            your_rests.append(name)
        else:
            your_rests.append(rest)
    return your_rests

In [68]:
batch_size = 10
save_path = './project_models/model.ckpt'
save_path_val = './project_models/modelv.ckpt'
factors = [2,3,4,5]
decays = [0.1,0.01,0.001]

In [188]:
checks = np.nonzero(interactions_train)
check_these = checks[0]

In [189]:
model = MatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], n_factors=2)
loss_func = torch.nn.MSELoss()
reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.01)
for i in range(10):
    running_loss = 0.0
    correct = 0
    for m, batch in enumerate(get_batch(batch_size, interactions_train)):
        reg_loss_func.zero_grad()
        interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
        predictions = model(rows, cols)
        
        loss = loss_func(predictions, interactions)
        running_loss += np.sqrt(loss.data[0])*batch_size
          
        #an attempt to do some accuracy? 
        for u in range(len(rows)):
            user = rows.data[u]
            row = Variable(torch.LongTensor([np.long(user)]))
            #print ("user:" + str(user))
            where = np.where(check_these==user)
            index = where[0]
            if len(index) > 0:
                for c in range(len(index)):
                    rest = checks[1][index[c]]
                    #print (rest)
                    col = Variable(torch.LongTensor([np.long(rest)]))
                    prediction = model.predict(row,col)
                    val = np.around(prediction.data[0])
                    #print (val)
                if val == interactions_train[user,item]:
                    correct = correct + 1 
            else:
                next
                
        loss.backward()
    reg_loss_func.step()
    
    epoch_loss = running_loss/interactions_train.shape[0]
    print ('train_loss: ' + str(epoch_loss))
    accur = correct/len(check_these)
    print ('accuracy:' + str(accur))
                                    
    if epoch_loss < best_train_loss:
        best_train_loss = epoch_loss
        print ('checkpoint model with train loss: ' + str(best_train_loss))
        checkpoint_model(epoch_loss, model, save_path)
val_loss = run_validation(model, interactions_test, batch_size, save_path)
print ('val loss: ' + str(val_loss))
        
if val_loss < best_val_loss:
    best_val_loss = val_loss
    print ('checkpointing_model with val loss: ' + str(best_val_loss))
    checkpoint_model(val_loss, model, save_path_val)
recs = generate_predictions(model, interactions_full, save_path_val)

/opt/conda/lib/python3.6/site-packages/scipy/sparse/compressed.py:214: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  ", try using != instead.", SparseEfficiencyWarning)


train_loss: 1.5605679748281682
accuracy:0.05794947994056464
train_loss: 1.5578844770283533
accuracy:0.06092124814264487
train_loss: 1.5653310283894912
accuracy:0.06240713224368499
train_loss: 1.5519863214577483
accuracy:0.06092124814264487
train_loss: 1.5496073590500297
accuracy:0.06092124814264487
train_loss: 1.5658727881189936
accuracy:0.0638930163447251
train_loss: 1.542282315446676
accuracy:0.0638930163447251
train_loss: 1.5624804967641204
accuracy:0.06092124814264487
train_loss: 1.5485956780220083
accuracy:0.06092124814264487
train_loss: 1.5619742337082352
accuracy:0.05794947994056464
val loss: 1.3235134789901182


In [247]:
for i in range(len(user_list)):
    print ("User: " + str(user_list[i]))
    results = give_top5(recs,rest_deets,user_list[i], user_list, item_list)
    print (results)

User: U1077
['Restaurant los Compadres', 'Restaurante la Estrella de Dima', 'TACOS CORRECAMINOS', 'Restaurant Oriental Express', 'El Rincon de San Francisco']
User: U1068
['Cafe Chaires', 'la Cochinita Pibil Restaurante Yucateco', 'Restaurant la Chalita', 'Restaurante y Pescaderia Tampico', 'Gorditas Dona Tota']
User: U1067
['carnitas mata calle Emilio Portes Gil', 'carnitas_mata', 'Hamburguesas Valle Dorado', 'Dominos Pizza', 'Cenaduria El RincÃ³n de Tlaquepaque']
User: U1103
['Cafe Chaires', 'Restaurant la Chalita', 'Restaurant los Pinos', 'Mariscos Tia Licha', 'la Cochinita Pibil Restaurante Yucateco']
User: U1107
['Dominos Pizza', 'carnitas_mata', 'la Cochinita Pibil Restaurante Yucateco', 'Hamburguesas Valle Dorado', 'Cenaduria El RincÃ³n de Tlaquepaque']
User: U1044
['El Rincon de San Francisco', 'Restaurant Orizatlan', 'TACOS CORRECAMINOS', 'Log Yin', 'la parroquia']
User: U1070
['Hamburguesas Valle Dorado', 'vips', 'don burguers', 'VIPS', 'el pueblito']
User: U1031
['Restaurant

['Restaurant los Compadres', 'Restaurante la Estrella de Dima', 'sirloin stockade', 'Tortas Locas Hipocampo', 'vips']
User: U1052
['carnitas_mata', 'carnitas mata calle Emilio Portes Gil', 'la Cochinita Pibil Restaurante Yucateco', 'Cafe Chaires', 'Cenaduria El RincÃ³n de Tlaquepaque']
User: U1036
['vips', 'sirloin stockade', 'Restaurant los Compadres', 'don burguers', 'puesto de gorditas']
User: U1038
['Cafe Chaires', 'la Cochinita Pibil Restaurante Yucateco', 'Restaurant la Chalita', 'Restaurante y Pescaderia Tampico', 'Sushi Itto']
User: U1071
['vips', 'sirloin stockade', 'don burguers', 'Hamburguesas Valle Dorado', 'puesto de gorditas']
User: U1114
['Dominos Pizza', 'carnitas mata calle Emilio Portes Gil', 'carnitas_mata', 'Hamburguesas Valle Dorado', 'Cenaduria El RincÃ³n de Tlaquepaque']
User: U1084
['vips', 'sirloin stockade', 'don burguers', 'Restaurant los Compadres', 'puesto de gorditas']
User: U1049
['Dominos Pizza', 'carnitas_mata', 'la Cochinita Pibil Restaurante Yucateco'

In [69]:
best_val_loss = 1000.0
for f in range(len(factors)):
    print ("FACTOR: " + str(factors[f]))
    for r in range(len(decays)):
        print ("Decay: " + str(decays[r]))
        #best_val_loss = 1000.0
        best_train_loss = 1000.0
        model = MatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], n_factors=factors[f])
        loss_func = torch.nn.MSELoss()
        reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=decays[r])
        for i in range(10):
            running_loss = 0.0
            for m, batch in enumerate(get_batch(batch_size, interactions_train)):
                reg_loss_func.zero_grad()
                interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
                predictions = model(rows, cols)
        
                loss = loss_func(predictions, interactions)
                running_loss += np.sqrt(loss.data[0])*batch_size
        
                loss.backward()
            reg_loss_func.step()
    
            epoch_loss = running_loss/interactions_train.shape[0]
            print ('train_loss: ' + str(epoch_loss))
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
                print ('checkpoint model with train loss: ' + str(best_train_loss))
                checkpoint_model(epoch_loss, model, save_path)
        val_loss = run_validation(model, interactions_test, batch_size, save_path)
        print ('val loss: ' + str(val_loss))
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print ('checkpointing_model with val loss: ' + str(best_val_loss))
            checkpoint_model(val_loss, model, save_path_val)

FACTOR: 2
Decay: 0.1
train_loss: 1.5944808869858995
checkpoint model with train loss: 1.5944808869858995
train_loss: 1.623557091184269
train_loss: 1.6205570596171173
train_loss: 1.6245267126973941
train_loss: 1.613516873289169
train_loss: 1.612642877229853
train_loss: 1.6190346868838994
train_loss: 1.5944087244184026
checkpoint model with train loss: 1.5944087244184026
train_loss: 1.6322436775974964
train_loss: 1.6136575780852476
val loss: 1.5465395476673471
checkpointing_model with val loss: 1.5465395476673471
Decay: 0.01
train_loss: 1.5953662991902373
checkpoint model with train loss: 1.5953662991902373
train_loss: 1.5771877611874037
checkpoint model with train loss: 1.5771877611874037
train_loss: 1.5941715875613185
train_loss: 1.5900937623168154
train_loss: 1.5745849226930797
checkpoint model with train loss: 1.5745849226930797
train_loss: 1.596649105678263
train_loss: 1.5417688583551468
checkpoint model with train loss: 1.5417688583551468
train_loss: 1.5503193629982224
train_loss: 

In [65]:
best_val_loss = 1000.0
for f in range(len(factors)):
    print ("FACTOR: " + str(factors[f]))
    best_train_loss = 1000.0
    model = MatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], n_factors=factors[f])
    loss_func = torch.nn.MSELoss()
    reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6)
    for i in range(10):
        running_loss = 0.0
        for m, batch in enumerate(get_batch(batch_size, interactions_train)):
            reg_loss_func.zero_grad()
            interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
            rows = Variable(torch.LongTensor(batch))
            cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
            predictions = model(rows, cols)
        
            loss = loss_func(predictions, interactions)
            running_loss += np.sqrt(loss.data[0])*batch_size
        
            loss.backward()
        reg_loss_func.step()
    
        epoch_loss = running_loss/interactions_train.shape[0]
        print ('train_loss: ' + str(epoch_loss))
        if epoch_loss < best_train_loss:
            best_train_loss = epoch_loss
            print ('checkpoint model with train loss: ' + str(best_train_loss))
            checkpoint_model(epoch_loss, model, save_path)
    val_loss = run_validation(model, interactions_test, batch_size, save_path)
    print ('val loss: ' + str(val_loss))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val)

FACTOR: 2
train_loss: 1.5707202464429557
checkpoint model with train loss: 1.5707202464429557
train_loss: 1.5824273866824945
train_loss: 1.5648295093836064
checkpoint model with train loss: 1.5648295093836064
train_loss: 1.5805239945826095
train_loss: 1.5798726691005682
train_loss: 1.5829294225433082
train_loss: 1.574358054092937
train_loss: 1.577532871097914
train_loss: 1.5734255132503463
train_loss: 1.5865383812094975
val loss: 1.5077145491273105
checkpointing_model with val loss: 1.5077145491273105
FACTOR: 3
train_loss: 1.8874713712047835
checkpoint model with train loss: 1.8874713712047835
train_loss: 1.8866474209950796
checkpoint model with train loss: 1.8866474209950796
train_loss: 1.8766574137908334
checkpoint model with train loss: 1.8766574137908334
train_loss: 1.8869124085398667
train_loss: 1.8736790246597375
checkpoint model with train loss: 1.8736790246597375
train_loss: 1.8706649137803013
checkpoint model with train loss: 1.8706649137803013
train_loss: 1.8767875522176953
t

In [58]:
mean = np.mean(interactions_train)
mean

0.10602006688963211

In [61]:
save_path_b = './project_models/bmodel.ckpt'
save_path_val_b = './project_models/bmodelv.ckpt'

In [62]:
best_val_loss = 1000.0
for f in range(len(factors)):
    print ("FACTOR: " + str(factors[f]))
    for r in range(len(decays)):
        print ("Decay: " + str(decays[r]))
        #best_val_loss = 1000.0
        best_train_loss = 1000.0
        model = BiasedMatrixFactorization(interactions_train.shape[0], interactions_train.shape[1], mean, n_factors=factors[f])
        loss_func = torch.nn.MSELoss()
        reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=decays[r])
        for i in range(10):
            running_loss = 0.0
            for m, batch in enumerate(get_batch(batch_size, interactions_train)):
                reg_loss_func.zero_grad()
                interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
                predictions = model(rows, cols)
        
                loss = loss_func(predictions, interactions)
                running_loss += np.sqrt(loss.data[0])*batch_size
        
                loss.backward()
            reg_loss_func.step()
    
            epoch_loss = running_loss/interactions_train.shape[0]
            print ('train_loss: ' + str(epoch_loss))
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
                print ('checkpoint model with train loss: ' + str(best_train_loss))
                checkpoint_model(epoch_loss, model, save_path_b)
        val_loss = run_validation(model, interactions_test, batch_size, save_path_b)
        print ('val loss: ' + str(val_loss))
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print ('checkpointing_model with val loss: ' + str(best_val_loss))
            checkpoint_model(val_loss, model, save_path_val_b)

FACTOR: 2
Decay: 0.1
train_loss: 2.1010208276268254
checkpoint model with train loss: 2.1010208276268254
train_loss: 2.1146876716491048
train_loss: 2.0970861322633976
checkpoint model with train loss: 2.0970861322633976
train_loss: 2.078513150762388
checkpoint model with train loss: 2.078513150762388
train_loss: 2.1082202126145937
train_loss: 2.0845825822209516
train_loss: 2.0843566854073576
train_loss: 2.0829738470832306
train_loss: 2.0709269537624473
checkpoint model with train loss: 2.0709269537624473
train_loss: 2.0595321550594043
checkpoint model with train loss: 2.0595321550594043
val loss: 2.009003324429658
checkpointing_model with val loss: 2.009003324429658
Decay: 0.01
train_loss: 2.1061262155352676
checkpoint model with train loss: 2.1061262155352676
train_loss: 2.125864896406354
train_loss: 2.107695877905106
train_loss: 2.1104984715786843
train_loss: 2.1168955440680097
train_loss: 2.0904834445989957
checkpoint model with train loss: 2.0904834445989957
train_loss: 2.100311221

In [66]:
best_val_loss = 1000.0
for f in range(len(factors)):
    print ("FACTOR: " + str(factors[f]))
    best_train_loss = 1000.0
    model = BiasedMatrixFactorization(interactions_train.shape[0], interactions_train.shape[1],mean, n_factors=factors[f])
    loss_func = torch.nn.MSELoss()
    reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6)
    for i in range(10):
        running_loss = 0.0
        for m, batch in enumerate(get_batch(batch_size, interactions_train)):
            reg_loss_func.zero_grad()
            interactions = Variable(torch.FloatTensor(interactions_train[batch,:].toarray()))
            rows = Variable(torch.LongTensor(batch))
            cols = Variable(torch.LongTensor(np.arange(interactions_train.shape[1])))
            predictions = model(rows, cols)
        
            loss = loss_func(predictions, interactions)
            running_loss += np.sqrt(loss.data[0])*batch_size
        
            loss.backward()
        reg_loss_func.step()
    
        epoch_loss = running_loss/interactions_train.shape[0]
        print ('train_loss: ' + str(epoch_loss))
        if epoch_loss < best_train_loss:
            best_train_loss = epoch_loss
            print ('checkpoint model with train loss: ' + str(best_train_loss))
            checkpoint_model(epoch_loss, model, save_path)
    val_loss = run_validation(model, interactions_test, batch_size, save_path)
    print ('val loss: ' + str(val_loss))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val)

FACTOR: 2
train_loss: 2.0601148269819056
checkpoint model with train loss: 2.0601148269819056
train_loss: 2.078074929939537
train_loss: 2.071905343596753
train_loss: 2.075484468429301
train_loss: 2.092951358020045
train_loss: 2.0661176163857187
train_loss: 2.0865121634017103
train_loss: 2.0831317007630457
train_loss: 2.0904322533952024
train_loss: 2.0960979413224243
val loss: 2.0547025545466773
checkpointing_model with val loss: 2.0547025545466773
FACTOR: 3
train_loss: 2.4422406997496546
checkpoint model with train loss: 2.4422406997496546
train_loss: 2.436855214838995
checkpoint model with train loss: 2.436855214838995
train_loss: 2.427085590662223
checkpoint model with train loss: 2.427085590662223
train_loss: 2.4439370718014892
train_loss: 2.444650610989361
train_loss: 2.4265913143154667
checkpoint model with train loss: 2.4265913143154667
train_loss: 2.438623116704
train_loss: 2.4389246554422184
train_loss: 2.4368984165790972
train_loss: 2.417333068248689
checkpoint model with trai